In [ ]:
import numpy as np
import time
from sklearn.metrics import accuracy_score
import prometheus_client as prom
from flask import Flask, request, jsonify


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load Iris dataset
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Initial Model Accuracy: {accuracy:.4f}")


In [ ]:
# Define Prometheus metrics
model_accuracy = prom.Gauge('model_accuracy', 'Accuracy of the deployed model')
prediction_latency = prom.Summary('prediction_latency', 'Latency of predictions')

# Update accuracy metric
model_accuracy.set(accuracy)


In [ ]:
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    start_time = time.time()

    # Parse input features
    input_data = request.json['features']
    input_array = np.array(input_data).reshape(1, -1)

    # Make prediction
    prediction = model.predict(input_array)
    class_name = iris.target_names[prediction[0]]

    # Measure latency
    latency = time.time() - start_time
    prediction_latency.observe(latency)

    return jsonify({"predicted_class": class_name, "latency": latency})

@app.route('/metrics')
def metrics():
    return prom.generate_latest(prom.REGISTRY)

if __name__ == '__main__':
    prom.start_http_server(8000)  # Start Prometheus on port 8000
    app.run(debug=True, port=5000)


In [ ]:
python app.py


In [ ]:
curl -X POST -H "Content-Type: application/json" -d '{"features": [5.1, 3.5, 1.4, 0.2]}' http://127.0.0.1:5000/predict


In [ ]:
http://127.0.0.1:8000/metrics


In [ ]:
sudo apt-get install grafana


In [ ]:
from scipy.stats import ks_2samp

# Simulate new data with drift
new_data = X_test + np.random.normal(0, 0.5, X_test.shape)

# Detect drift using Kolmogorov-Smirnov test
for i, feature in enumerate(iris.feature_names):
    stat, p_value = ks_2samp(X_test[:, i], new_data[:, i])
    if p_value < 0.05:
        print(f"Feature '{feature}' shows significant drift (p-value: {p_value:.4f})")


In [ ]:
groups:
  - name: model_monitoring
    rules:
    - alert: AccuracyDrop
      expr: model_accuracy < 0.8
      for: 2m
      labels:
        severity: warning
      annotations:
        summary: "Model accuracy dropped below 80%"
